In [26]:
# Ferromagnetic Transverse-Interaction QA vs. covenvtional Transverse-Field QA
# Exact minimum-gap benchmark, L = 9, 10 disorder seeds
# Shahriyar Dadgar — June 2025
#
# ────────────────────────────────────────────────────────────────────────────
# One script, no external files:
#   • Builds problem Hamiltonian H_z (Random spin glass - SK model: Gaussian J_ij and longitudinal fields)
#   • Builds transverse-field driver H_x (–Σ_i σ_i^x)
#   • Builds ferromagnetic transverse-interaction driver H_xx  (–Σ_ij σ_i^x σ_j^x / L)
#   • Sweeps anneal parameter s ∈ [0,1] with step dt
#   • For each s, computes the lowest two eigenvalues via SciPy eigsh
#   • Records the minimum spectral gap for each schedule

import os, numpy as np, scipy.sparse as sp, scipy.sparse.linalg as spla
from multiprocessing import Pool, cpu_count
os.environ["OMP_NUM_THREADS"] = "1"

def get_spin(state, position):
    '''Returns the spin value at the specified position.'''
    return 1 if (state >> position) & 1 == 0 else -1

#problem hamiltonian:
def H_z(L, seed):
    np.random.seed(seed)
    mean, var = 0, 1/L
    std = np.sqrt(var)
    dim = 2**L
    H = sp.lil_matrix((dim, dim), dtype=np.float32)
    for state in range(dim):
        diag = 0
        for i in range(L):
            for j in range(i+1, L):
                diag += np.random.normal(mean, std) * get_spin(state, i) * get_spin(state, j)
        diff = bin(state).count('1') - (L - bin(state).count('1'))
        diag += np.random.normal(mean, std) * diff
        H[state, state] = diag
    return H.tocsc()

#transverse field:
def H_x(L):
    dim = 2**L
    H = sp.lil_matrix((dim, dim), dtype=np.float32)
    for state in range(dim):
        for site in range(L):
            H[state, state ^ (1 << site)] = -1
    return H.tocsc()

#ferromagnetic transverse interaction (FTI):
def H_xx(L):
    dim = 2**L
    H = sp.lil_matrix((dim, dim), dtype=np.float32)
    for state in range(dim):
        for i in range(L):
            for j in range(L):
                state_prime = state ^ (1 << i)
                state_dprime = state_prime ^ (1 << j)
                H[state, state_dprime] -= 1/L
    return H.tocsc()

def time_step_multiprocessing(params):
    s, r = params
    lmbd = s**r
    #FTI schedule
    H = s*lmbd*Hz + s*(1-lmbd)*Hxx + (1-s)*Hx
    ev, vec = spla.eigsh(H, k = 2, which='SA')
    gap_FTI = ev[1] - ev[0]
    #conventional quantum annealing schedule:
    H = s*Hz + (1-s)*Hx
    ev, vec = spla.eigsh(H, k = 2, which='SA')
    gap_QA = ev[1] - ev[0]
    return gap_FTI, gap_QA


if __name__ == '__main__':
    L = 9           # number of spins
    r = 2           # λ(s) exponent
    dt = 0.005      # annealing step
    sample_size = 10    # disorder seeds

    Hx = H_x(L)
    Hxx = H_xx(L)

    SEEDS = list(range(sample_size))
    for seed in SEEDS:
        Hz = H_z(L, seed)
        s_values =np.linspace(0, 1 , int(1/dt))
        params = [(float(s), r)
                    for s in s_values]
        # Use a pool of worker processes to handle time steps in parallel
        with Pool(processes=cpu_count()) as pool:
            results = pool.map(time_step_multiprocessing, params)

        FTI_results, QA_results = zip(*results)
        FTI_results = np.array(FTI_results)
        QA_results = np.array(QA_results)
        print(seed, FTI_results.min(), QA_results.min(), FTI_results.min()/QA_results.min())

0 0.26656628 0.2035265 1.3097374
1 0.25060177 0.19884491 1.2602875
2 0.27669764 0.3078022 0.8989463
3 0.19106483 0.16078043 1.1883588
4 0.20957994 0.16467333 1.2727013
5 0.21501446 0.17434835 1.2332463
6 0.30581808 0.26413393 1.1578145
7 0.19827557 0.11917782 1.6636952
8 0.16293287 0.10158157 1.6039609
9 0.018552303 0.001057148 17.54939


#### Summary for L = 7 – 17 (with 100 disorder realizations/hard problems, annealing step set to 0.001 and r equals to 2.)



| L | Fraction of instances with min_ΔFTI > min_ΔQA | mean ratio of minimum gaps for those instances |
|---|------------------|---------------------|
|  7 |         0.88 |         1.66 |
|  8 |         0.85 |         2.69 |
|  9 |         0.80 |         2.57 |
| 10 |         0.82 |         3.44 |
| 11 |         0.82 |         4.09 |
| 12 |         0.82 |         2.49 |
| 13 |         0.84 |         2.47 |
| 14 |         0.83 |         5.73 |
| 15 |         0.80 |         5.69 |
| 16 |         0.86 |         4.58 |
| 17 |         0.73 |         5.80 |

*(Values loaded offline and pasted for quick viewing.)*

Important caveats:
- Even though the mean ratio seems to hold the advantage of FTI for the large Ls, the medians of the ratios remain only slightly above one (about 10 percent).
- The minimum gap for FTI seems to be drastically decreasing with L (most likely exponentially which is the major generic bottle neck for quantum annealing algorithms).

Notice that these results are based on standard SK model, as opposed to the results in Phys. Rev. B 95, 184416 (2017) where they set the variance of the Gaussian distribution of the random values constructing the problem Hamiltonian to 1 (not 1/L).